In [1]:
import requests

headers = {
    "X-API-Key": "98f7e5eeb14e29b01fbb62fea2bb4b6a6e5dca0fe09bcff6d2a73c824fbaaa33"
}


r = requests.get("https://api.openaq.org/v3/locations", headers=headers)
print(r.status_code)
print(r.json().keys())



StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 3, Finished, Available, Finished)

200
dict_keys(['meta', 'results'])


In [2]:
params = {
    "bbox": "-74.2591,40.4774,-73.7004,40.9176",
    "limit": 100
}

r = requests.get(
    "https://api.openaq.org/v3/locations",
    headers=headers,
    params=params
)

payload = r.json()
locations = payload["results"]

print("Locations found:", len(locations))


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 4, Finished, Available, Finished)

Locations found: 55


In [3]:
from pprint import pprint
pprint(locations[0]["sensors"])


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 5, Finished, Available, Finished)

[{'id': 671,
  'name': 'o3 ppm',
  'parameter': {'displayName': 'O₃', 'id': 10, 'name': 'o3', 'units': 'ppm'}},
 {'id': 673,
  'name': 'pm25 µg/m³',
  'parameter': {'displayName': 'PM2.5',
                'id': 2,
                'name': 'pm25',
                'units': 'µg/m³'}}]


In [4]:
print(locations[0].keys())


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 6, Finished, Available, Finished)

dict_keys(['id', 'name', 'locality', 'timezone', 'country', 'owner', 'provider', 'isMobile', 'isMonitor', 'instruments', 'sensors', 'coordinates', 'licenses', 'bounds', 'distance', 'datetimeFirst', 'datetimeLast'])


In [6]:
url = f"https://api.openaq.org/v3/sensors/{671}/measurements"

params = {
    "limit": 10
}

r = requests.get(url, headers=headers, params=params)

print("Status:", r.status_code)
print(r.json().keys())


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 8, Finished, Available, Finished)

Status: 200
dict_keys(['meta', 'results'])


In [7]:
params = {
    "date_from": "2025-01-01",
    "date_to": "2025-12-31",
    "limit": 100
}

r = requests.get(url, headers=headers, params=params)

data = r.json().get("results", [])
print("Rows:", len(data))

if data:
    print("Parameter:", data[0]["parameter"])


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 9, Finished, Available, Finished)

Rows: 100
Parameter: {'id': 10, 'name': 'o3', 'units': 'ppm', 'displayName': None}


In [8]:
sensors = []

for loc in locations:
    for s in loc.get("sensors", []):
        sensors.append({
            "sensor_id": s["id"],
            "parameter": s["parameter"],
            "location_name": loc["name"]
        })

print("Total sensors:", len(sensors))



StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 10, Finished, Available, Finished)

Total sensors: 194


In [9]:
active_sensors = []

for s in sensors:
    url = f"https://api.openaq.org/v3/sensors/{s['sensor_id']}/measurements"

    r = requests.get(
        url,
        headers=headers,
        params={
            "date_from": "2025-01-01",
            "date_to": "2025-12-31",
            "limit": 1
        }
    )

    if r.status_code == 200 and r.json().get("results"):
        active_sensors.append(s)

print("Active sensors in 2025:", len(active_sensors))



StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 11, Finished, Available, Finished)

Active sensors in 2025: 61


In [10]:
sensor_id = active_sensors[0]["sensor_id"]

url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements"

r = requests.get(
    url,
    headers=headers,
    params={
        "date_from": "2025-01-01",
        "date_to": "2025-12-31",
        "limit": 1
    }
)

print("Status:", r.status_code)
print("Results:", len(r.json().get("results", [])))


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 12, Finished, Available, Finished)

Status: 200
Results: 1


In [11]:
print(locations[0]["datetimeLast"])


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 13, Finished, Available, Finished)

{'utc': '2026-01-07T12:00:00Z', 'local': '2026-01-07T07:00:00-05:00'}


In [12]:
sensor_id = active_sensors[0]["sensor_id"]

url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements"

r = requests.get(
    url,
    headers=headers,
    params={
        "date_from": "2025-01-01",
        "date_to": "2025-12-31",
        "limit": 10
    }
)

print("Raw rows:", len(r.json().get("results", [])))


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 14, Finished, Available, Finished)

Raw rows: 10


In [13]:
url = f"https://api.openaq.org/v3/sensors/{sensor_id}/averages"

r = requests.get(
    url,
    headers=headers,
    params={
        "period": "day",
        "date_from": "2025-01-01",
        "date_to": "2025-12-31"
    }
)

print("Averages rows:", len(r.json().get("results", [])))


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 15, Finished, Available, Finished)

Averages rows: 0


In [14]:
def extract_datetime(row):
    if "date" in row and isinstance(row["date"], dict):
        return row["date"].get("utc")
    if "datetime" in row:
        return row["datetime"]
    if "observedAt" in row:
        return row["observedAt"]
    return None


rows = []

for s in active_sensors:
    url = f"https://api.openaq.org/v3/sensors/{s['sensor_id']}/measurements"

    r = requests.get(
        url,
        headers=headers,
        params={
            "date_from": "2025-01-01",
            "date_to": "2025-12-31",
            "limit": 200
        }
    )

    for row in r.json().get("results", []):
        dt = extract_datetime(row)
        if dt is None:
            continue

        rows.append({
            "datetime": dt,
            "parameter": row.get("parameter"),
            "value": row.get("value"),
            "unit": row.get("unit"),
            "sensor_id": s["sensor_id"],
            "location_name": s["location_name"]
        })



StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 16, Finished, Available, Finished)

In [19]:
def sensor_has_2025(sensor_id):
    url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements"

    r = requests.get(
        url,
        headers=headers,
        params={
            "limit": 1,
            "sort": "desc"   # последний замер
        }
    )

    results = r.json().get("results", [])
    if not results:
        return False

    last_dt = results[0]["period"]["datetimeTo"]["utc"]
    return last_dt.startswith("2025")


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 21, Finished, Available, Finished)

In [30]:
valid_sensors = []

for loc in locations:
    for s in loc["sensors"]:
        if sensor_has_2025(s["id"]):
            valid_sensors.append({
                "sensor_id": s["id"],
                "location": loc["name"]
            })

len(valid_sensors)


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 32, Finished, Available, Finished)

27

In [37]:
rows = []

MAX_PAGES = 5   

for s in valid_sensors:
    sensor_id = s["sensor_id"]
    location_name = s["location"]

    url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements"

    for page in range(1, MAX_PAGES + 1):
        r = requests.get(
            url,
            headers=headers,
            params={
                "limit": 100,
                "page": page,
                "sort": "desc"
            }
        )

        results = r.json().get("results", [])
        if not results:
            break

        for row in results:
            dt = row["period"]["datetimeTo"]["utc"]

            if not dt.startswith("2025"):
                continue

            rows.append({
                "datetime": dt,
                "parameter": row["parameter"]["name"],
                "value": row["value"],
                "unit": row["parameter"]["units"],
                "sensor_id": sensor_id,
                "location": location_name
            })


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 39, Finished, Available, Finished)

In [38]:
len(rows)


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 40, Finished, Available, Finished)

6100

In [39]:
set(r["parameter"] for r in rows)


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 41, Finished, Available, Finished)

{'pm1', 'pm25', 'relativehumidity', 'temperature', 'um003'}

In [40]:
min(r["datetime"] for r in rows), max(r["datetime"] for r in rows)


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 42, Finished, Available, Finished)

('2025-02-11T18:00:00Z', '2025-06-08T20:00:00Z')

In [41]:
import pandas as pd

df = pd.DataFrame(rows)
df["datetime"] = pd.to_datetime(df["datetime"])


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 43, Finished, Available, Finished)

In [42]:
pollutants = {"pm1", "pm25", "um003"}
df_pollution = df[df["parameter"].isin(pollutants)]


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 44, Finished, Available, Finished)

In [44]:
daily = (
    df_pollution
    .set_index("datetime")
    .groupby("parameter")
    .resample("D")["value"]
    .mean()
    .reset_index()
)



StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 46, Finished, Available, Finished)

In [45]:
daily.head()


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 47, Finished, Available, Finished)

,parameter,datetime,value
0,pm1,2025-02-11 00:00:00+00:00,8.024611
1,pm1,2025-02-12 00:00:00+00:00,14.728221
2,pm1,2025-02-13 00:00:00+00:00,8.453886
3,pm1,2025-02-14 00:00:00+00:00,2.533100
4,pm1,2025-02-15 00:00:00+00:00,4.869012


In [50]:
df_bronze = df_pollution.copy()


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 52, Finished, Available, Finished)

In [49]:
import os

bronze_path = "/lakehouse/default/Files/bronze/openaq/nyc_2025"
os.makedirs(bronze_path, exist_ok=True)

df_bronze.to_parquet(
    f"{bronze_path}/measurements.parquet",
    index=False
)


StatementMeta(, b9222d36-1884-4825-a8ef-25af3df78621, 51, Finished, Available, Finished)